In [1]:
from astroquery.ned import Ned 
from astroquery.sdss import SDSS 
#import astroquery as aq
from astropy import coordinates as coords   #Conversion, sistemas y marcos de referencia
from astropy import units as u   #Conversion y desarrollo de operaciones aritméticas de instancias Quantity
from astropy.io import fits   
from astropy.table import Table
import numpy as np
#import sys

/usr/lib/python2.7/site-packages/astroquery/sdss/__init__.py:33: UserWarning: Experimental: SDSS has not yet been refactored to have its API match the rest of astroquery (but it's nearly there).
  warnings.warn("Experimental: SDSS has not yet been refactored to have its API "


In [2]:
newList = Table.read("newObj.txt", format = 'ascii')
len(newList)

200

In [3]:
newResultListDT=Table(names=('ObjID','ra','dec','z', 'isFits'),dtype=('S25', 'f12', 'f12','f8', 'i1'))
ListaGalaxias=newList['col3']
for i in range(len(ListaGalaxias)):
    name = newList['col3'][i]
    # resultNED= Ned.query_object(ListaGalaxias[i])
    RA = newList['col4'][i]  #*u.degree   #resultNED['RA(deg)'][0]*u.degree   
    DEC = newList['col5'][i] #*u.degree   #resultNED['DEC(deg)'][0]*u.degree
    z = newList['col7'][i]    #resultNED['Redshift'][0]
    pos=coords.SkyCoord(ra=RA, dec=DEC, frame='icrs', unit='deg')  #Cordenadas en sistema ICRS
    try:
        xid = SDSS.query_region(pos, spectro=True)
        if xid is None:
            print i, str(name)+': Not found'
        else:
            sp = SDSS.get_spectra(matches=xid)
            hdulist = fits.HDUList(sp[0])
            total= hdulist[1].data
            linlam=np.exp(total['loglam']*np.log(10))
            lambd = linlam/(1+z)
            if lambd[0]<3700 and lambd[-1]>6800:
                print i, str(name)+': Found '+str(len(sp))+' spectra'
                hdulist.writeto(str(ListaGalaxias[i])+'.fits',clobber=True)
                newResultListDT.add_row([ListaGalaxias[i],pos.ra*u.degree,pos.dec*u.degree,z, 1])
    except: # HTTPError:
        # e = sys.exc_info()[0]
        # write_to_page( "<p>Error: %s</p>" % e )
        continue

newResultListDT.write('newListSDSS.tbl', format='ascii')

0 SDSS J000011.60+010150.5: Found 1 spectra
1 2MASX J00002629+0035503: Found 1 spectra
2 HELLAS2XMM 26900022: Not found
3 KUG 2358+330: Not found
4 2MASX J00013021-0014459: Found 1 spectra
5 CGCG 517-014: Not found
6 SDSS J000259.42+001815.6: Found 1 spectra
7 UM 016: Not found
8 SDSS J000318.21+004844.3: Found 1 spectra
9 GALEXMSC J000319.43+212946.0: Not found
10 2MASX J00034461+0034294: Found 1 spectra
11 2MASX J00043193-0114116: Found 1 spectra
12 2MASX J00065546-0039190: Found 1 spectra
13 2MASX J00070979+0104004: Found 1 spectra
14 2MASX J00073554+0047455: Found 1 spectra
15 2MASX J00075612-0033005: Found 1 spectra
17 2MASSi J0008108+135452: Found 1 spectra
18 SDSS J000819.72-000002.7: Found 1 spectra
19 2MASX J00082041+4037560: Not found
20 2MASX J00090793+1427558: Found 1 spectra
21 SDSS J000908.27-011013.8: Found 1 spectra
22 2MASX J00100554+0040247: Found 1 spectra
23 SDSS J001014.86+000820.8: Found 1 spectra
24 2MASX J00102892+0103010: Found 1 spectra
25 ESO 538- G 025: Not 

In [5]:
#ListaGalaxias[i]
len(newResultListDT)

129

In [14]:
#        10h23m30.6s +19d51m54s     0.003859
linlam

array([ 3795.77148438,  3796.64746094,  3797.52026367, ...,  9198.140625  ,
        9200.25488281,  9202.37890625], dtype=float32)

In [13]:
pos.ra*u.degree

<Quantity 0.28804166666666664 deg2>